<a href="https://colab.research.google.com/github/umaabhyankar/AQI-Predictor/blob/main/feature_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
!pip install geopy folium streamlit-folium --q

In [33]:
import datetime
import time
import requests
import pandas as pd
import numpy as np
import json

from feature_funcs import *

import warnings
warnings.filterwarnings("ignore")

In [15]:
today = datetime.date.today()

In [69]:
response_API = requests.get('https://api.waqi.info/feed/solapur/?token=6e62f66525787498e49dd5df9d7a3993ff183759')

In [70]:
data = response_API.text

In [71]:
df_data = json.loads(data)

In [72]:
df_forecast = pd.DataFrame(columns = ['date', 'average', 'pollutant'])
for pollutant, values in df_data['data']['forecast']['daily'].items():
  df_forecast_x = pd.DataFrame()
  df_forecast_x['date'] = [i['day'] for i in values]
  df_forecast_x['average'] = [i['avg'] for i in values]
  df_forecast_x['pollutant'] = [pollutant] * len(values)
  df_forecast = df_forecast.append(df_forecast_x)

In [73]:
table = pd.pivot_table(df_forecast, values ='average', index =['date'],
                         columns =['pollutant'], aggfunc = np.sum)

In [74]:
table.reset_index(inplace = True)

In [75]:
table = table[['date','pm25','pm10','o3']]

In [76]:
table['no2'] = [0]*len(table)
table['so2'] = [0]*len(table)
table['co'] = [0]*len(table)

In [77]:
table['date'] = pd.to_datetime(table['date'])

In [78]:
table.fillna(0, inplace=True)

In [79]:
table['no2'] = table['no2'].astype('float')
table['so2'] = table['so2'].astype('float')
table['co'] = table['co'].astype('float')

In [80]:
table['n_count'] = np.count_nonzero(table[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']], axis=1)
table['n_count'] = table['n_count'].astype('float')

In [81]:
pm10_si_list = []
pm25_si_list = []
so2_si_list = []
no2_si_list = []
co_si_list = []
o3_si_list = []

for i in table['pm10']:
    pm10_si_list.append(pm10_sub_index(i))

for i in table['pm25']:
    pm25_si_list.append(pm25_sub_index(i))

for i in table['o3']:
    o3_si_list.append(o3_sub_index(i))

for i in table['no2']:
    no2_si_list.append(no2_sub_index(i))

for i in table['so2']:
    so2_si_list.append(so2_sub_index(i))

for i in table['co']:
    co_si_list.append(co_sub_index(i))

In [82]:
table['pm10_si'] = pm10_si_list
table['pm25_si'] = pm25_si_list
table['o3_si'] = o3_si_list
table['no2_si'] = no2_si_list
table['so2_si'] = so2_si_list
table['co_si'] = co_si_list

In [83]:
table['AQI'] = table[['pm10_si', 'pm25_si', 'o3_si', 'no2_si', 'so2_si', 'co_si']].max(axis = 1)

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()

air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality',
    version = 1
)

In [ ]:
air_quality_fg.insert(table, write_options={"wait_for_job": False})